In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os
os.chdir("/content/drive/Shareddrives/hoa.lenghiem/NLP_Paraphrasing_Generator")

In [ ]:
# required libraries

import unicodedata
import string
import re
import random
import time
import datetime
import math

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence
import torchtext
from torchtext.legacy.datasets import TranslationDataset

import spacy
import numpy as np
from nltk.translate.bleu_score import corpus_bleu

# set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
import pandas as pd
file_path = 'merged_datasets.csv'
df = pd.read_csv(file_path)
df.drop(columns=['id'], inplace=True)
df.head()

,sentence1,sentence2,label
0,Một số người đang cùng nhau đá bóng trên cát n...,Một số người đang chơi bóng đá chân không ngoà...,1
1,Một cầu thủ ném bóng đang chuẩn bị ném bóng .,Khán giả đang theo dõi cú ném bóng của cầu thủ...,1
2,Một cầu thủ ném bóng áo đỏ đang chạy về phía g...,Những người đàn ông đang chơi bóng chày ở trên...,1
3,Một nam vận động viên tennis đang đưa vợt ra đ...,Một người đàn ông đang bước tới trước ra để đá...,1
4,Một nam vận động viên tennis đang nhón chân để...,Một người đàn ông đang nhón chân để đánh quả b...,1


In [ ]:
nonpara = pd.DataFrame(df.loc[df["label"] == 0])
nonpara.head()

,sentence1,sentence2,label
7,"Để bán được hàng thàng công , trước tiên bạn p...",Đó chính là lý do tại sao nhân viên bán hàng b...,0
8,Điều này cho phép bạn lựa chọn những công việc...,Ngày làm việc của bạn kết thúc và bạn tự nhủ r...,0
9,"Vì vậy , bạn hãy học cách nói “ không ” với nh...",Bạn cũng có chọn cách chỉnh sửa lại danh sách ...,0
10,"Để có được những thành tựu như ngày hôm nay , ...",Từng liên tục giữ thứ hạng cao nhất nước về tố...,0
11,"Không dừng lại ở bạn bè , thầy cô , mà ngay cả...",Bên cạnh một số bạn trẻ có những lời động viên...,0


In [ ]:
sentence1_col = nonpara['sentence1']
sentence2_col = nonpara['sentence2']
new_nonpara = pd.DataFrame(pd.concat([sentence1_col, sentence2_col], axis=0), columns=['src'])
new_nonpara.head()

,src
7,"Để bán được hàng thàng công , trước tiên bạn p..."
8,Điều này cho phép bạn lựa chọn những công việc...
9,"Vì vậy , bạn hãy học cách nói “ không ” với nh..."
10,"Để có được những thành tựu như ngày hôm nay , ..."
11,"Không dừng lại ở bạn bè , thầy cô , mà ngay cả..."


In [ ]:
new_nonpara.to_csv("test_paraphrase.csv", index=False)

In [ ]:
df = df.loc[df['label'] == 1]
df.head()

,sentence1,sentence2,label
0,Một số người đang cùng nhau đá bóng trên cát n...,Một số người đang chơi bóng đá chân không ngoà...,1
1,Một cầu thủ ném bóng đang chuẩn bị ném bóng .,Khán giả đang theo dõi cú ném bóng của cầu thủ...,1
2,Một cầu thủ ném bóng áo đỏ đang chạy về phía g...,Những người đàn ông đang chơi bóng chày ở trên...,1
3,Một nam vận động viên tennis đang đưa vợt ra đ...,Một người đàn ông đang bước tới trước ra để đá...,1
4,Một nam vận động viên tennis đang nhón chân để...,Một người đàn ông đang nhón chân để đánh quả b...,1


In [ ]:
df.shape

(5424, 3)

In [ ]:
df.drop(columns=["label"], inplace=True)

In [ ]:
df.to_csv("merged_datasets_paraphrased.csv", index=False)

In [ ]:
!pip install pyvi
!pip install https://gitlab.com/trungtv/vi_spacy/-/raw/master/vi_core_news_lg/dist/vi_core_news_lg-0.0.1.tar.gz

     |████████████████████████████████| 8.5 MB 4.7 MB/s 
     |████████████████████████████████| 743 kB 35.3 MB/s 
     |████████████████████████████████| 254.5 MB 39 kB/s 
     |████████████████████████████████| 6.3 MB 4.4 MB/s 
     |████████████████████████████████| 451 kB 34.1 MB/s 
     |████████████████████████████████| 628 kB 33.9 MB/s 
     |████████████████████████████████| 10.1 MB 35.4 MB/s 
     |████████████████████████████████| 42 kB 1.4 MB/s 
  Created wheel for vi-core-news-lg: filename=vi_core_news_lg-0.0.1-py3-none-any.whl size=254513617 sha256=47e5ec0e719750ef7a0a14492bd0f48139d2444c129883f5853930b67fae7809
  Stored in directory: /root/.cache/pip/wheels/e4/d9/90/dcbb25186a2c3335b17bc675f8dddcb43ecdfa4b400e8c91b0
Successfully built vi-core-news-lg
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation

In [ ]:
'''
tokenization code
'''

import spacy
spacy_vi = spacy.load('vi_core_news_lg')

def tokenize_vi(text):
    """
    Tokenizes Vietnamese text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_vi.tokenizer(text)]

'''
define field
'''
SRC = torchtext.legacy.data.Field(tokenize = tokenize_vi, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)
TRG = torchtext.legacy.data.Field(tokenize = tokenize_vi, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

In [ ]:
'''
load the data
'''
from torchtext.legacy import data
from torchtext.legacy import datasets

train_data = torchtext.legacy.data.TabularDataset(
    path='merged_datasets_paraphrased.csv', 
    format='csv', skip_header=True, fields=[('sentence1', SRC), 
                                            ('sentence2', TRG), 
                                            # ('label', LABEL)
                                            ],
                                            )

print(f"Number of training examples: {len(train_data.examples)}")

'''
build the vocabulary
'''
TRG.build_vocab(train_data, min_freq=2)
SRC.build_vocab(train_data, min_freq=2)
print(f"Unique tokens in source (vi) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (vi) vocabulary: {len(TRG.vocab)}")

'''
create the iterator
'''
train_iter = torchtext.legacy.data.BucketIterator(train_data, batch_size=16, device=device, sort_key=lambda x: len(x.sentence1), sort_within_batch=True)

Number of training examples: 5424
Unique tokens in source (vi) vocabulary: 2674
Unique tokens in target (vi) vocabulary: 2763


In [ ]:
type(train_iter)

torchtext.legacy.data.iterator.BucketIterator

In [ ]:
print(vars(train_data[0]))

{'sentence1': ['một_số', 'người', 'đang', 'cùng', 'nhau', 'đá', 'bóng', 'trên', 'cát', 'ngoài', 'biển', '.'], 'sentence2': ['một_số', 'người', 'đang', 'chơi', 'bóng_đá', 'chân_không', 'ngoài', 'bãi', 'biển', '.']}


In [ ]:
# save the field
import pickle
with open("SRC.Field","wb")as f:
     pickle.dump(SRC,f)
with open("TRG.Field","wb")as f:
     pickle.dump(TRG,f)

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim,n_layers, dropout):
        super().__init__()

        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dropout = dropout
        self.n_layers = n_layers

        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.lstm = nn.LSTM(emb_dim, enc_hid_dim, n_layers, dropout=dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, (hidden, cell) = self.lstm(embedded)
        
        return hidden, cell

class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, dec_hid_dim, n_layers, dropout):
        super().__init__()

        self.emb_dim = emb_dim
        self.output_dim = output_dim
        self.dec_hid_dim = dec_hid_dim
        self.n_layers = n_layers
        self.dropout = dropout

        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.lstm = nn.LSTM(emb_dim, dec_hid_dim, n_layers, dropout=dropout)
        self.fc_out = nn.Linear(dec_hid_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        
        input = input.unsqueeze(0)
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]    
        output, (hidden, cell) = self.lstm(embedded, (hidden, cell))
        
        # predicted shape is [batch_size, output_dim]
        prediction = self.fc_out(hidden.squeeze(0))
        
        return prediction, hidden, cell

class Seq2Seq(nn.Module):
    ''' 
    Args:
        encoder: A Encoder class instance.
        decoder: A Decoder class instance.
    '''
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        batch_size = trg.shape[1]
        max_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        # to store the outputs of the decoder
        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)

        hidden, cell = self.encoder(src)

        # first input to the decoder is the <sos> tokens
        input = trg[0, :]

        for t in range(1, max_len):
            output, hidden, cell = self.decoder(input, hidden, cell)
            outputs[t] = output
            use_teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.max(1)[1]
            input = (trg[t] if use_teacher_force else top1)

        # outputs is of shape [sequence_len, batch_size, output_dim]
        return outputs

In [ ]:
import os
'''
hyperparameters
'''
INPUT_DIM = len(SRC.vocab)
print(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
N_LAYERS = 1
LEARNING_RT = 0.001
N_EPOCHS = 50
CLIP = 1

'''
instantiate the model
'''
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, DEC_HID_DIM, N_LAYERS, DEC_DROPOUT)
model = Seq2Seq(enc, dec,device).to(device)
optimizer = optim.Adam(model.parameters(), lr = LEARNING_RT)
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
print('<pad> token index: ',TRG_PAD_IDX)
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

'''
initialize the model weights
'''
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
model.apply(init_weights)

'''
calculate the number of parameters
'''
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The model has {count_parameters(model):,} trainable parameters')

'''
compute the loss function with first training batch
'''
clip = 1
model.train()



for i, batch in enumerate(train_iter):

    src = batch.sentence1
    trg = batch.sentence2
    optimizer.zero_grad()

    output = model(src, trg)

    output_dim = output.shape[-1]

    output = output[1:].view(-1, output_dim)
    trg = trg[1:].view(-1)

    loss = criterion(output, trg)

    loss.backward()

    torch.nn.utils.clip_grad_norm_(model.parameters(), CLIP)

    optimizer.step()
    
    print('loss from first training batch')
    print(loss/src.shape[1])
    break

'''
Full training helper functions
'''

def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.sentence1
        trg = batch.sentence2
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        loss = criterion(output, trg)
        
        loss.backward()
      
        torch.nn.utils.clip_grad_norm_(model.parameters(), CLIP)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.sentence1
            trg = batch.sentence2

            output = model(src, trg, 0) #turn off teacher forcing

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

# convert index to text string
def convert_itos(convert_vocab, token_ids):
    list_string = []
    for i in token_ids:
        if i == convert_vocab.vocab.stoi['<eos>']:
            break
        else:
            token = convert_vocab.vocab.itos[i]
            list_string.append(token)
    return list_string
    

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

'''
kickstart full training
'''

best_valid_loss = float('inf')
for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iter, optimizer, criterion, CLIP)

    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    # Create checkpoint at end of each epoch
    state_dict_model = model.state_dict() 
    state = {
        'epoch': epoch,
        'state_dict': state_dict_model,
        'optimizer': optimizer.state_dict()
        }

    if not os.path.isdir("ckpt_seq2seq"):
        os.mkdir("ckpt_seq2seq")
    torch.save(state, "ckpt_seq2seq/seq2seq_"+str(epoch+1)+".pt")

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')


<pad> token index:  1
The model has 5,963,211 trainable parameters


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


loss from first training batch
tensor(0.4953, device='cuda:0', grad_fn=<DivBackward0>)
Epoch: 01 | Time: 0m 23s
	Train Loss: 4.780 | Train PPL: 119.073
Epoch: 02 | Time: 0m 22s
	Train Loss: 4.056 | Train PPL:  57.737
Epoch: 03 | Time: 0m 22s
	Train Loss: 3.742 | Train PPL:  42.172
Epoch: 04 | Time: 0m 22s
	Train Loss: 3.553 | Train PPL:  34.904
Epoch: 05 | Time: 0m 22s
	Train Loss: 3.410 | Train PPL:  30.273
Epoch: 06 | Time: 0m 22s
	Train Loss: 3.272 | Train PPL:  26.362
Epoch: 07 | Time: 0m 22s
	Train Loss: 3.178 | Train PPL:  23.995
Epoch: 08 | Time: 0m 22s
	Train Loss: 3.063 | Train PPL:  21.386
Epoch: 09 | Time: 0m 22s
	Train Loss: 2.986 | Train PPL:  19.802
Epoch: 10 | Time: 0m 22s
	Train Loss: 2.908 | Train PPL:  18.323
Epoch: 11 | Time: 0m 22s
	Train Loss: 2.835 | Train PPL:  17.024
Epoch: 12 | Time: 0m 22s
	Train Loss: 2.757 | Train PPL:  15.747
Epoch: 13 | Time: 0m 22s
	Train Loss: 2.669 | Train PPL:  14.423
Epoch: 14 | Time: 0m 22s
	Train Loss: 2.594 | Train PPL:  13.387
Epo

# Resuming

In [ ]:
'''
load fields saved during preprocessing
'''
with open("TRG.Field","rb") as f:
     TRG_saved = pickle.load(f)

with open("SRC.Field","rb") as f:
     SRC_saved = pickle.load(f)

'''
hyperparameters (ensure the following hyperparameters match with those used during training of the best model)
'''
INPUT_DIM = len(SRC_saved.vocab)
OUTPUT_DIM = len(TRG_saved.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
N_LAYERS = 1

'''
instantiate the model
'''
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, DEC_HID_DIM, N_LAYERS, DEC_DROPOUT)
model_best = Seq2Seq(enc, dec,device).to(device)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [ ]:
'''
load the checkpoint corresponding to the best epoch (usually epoch with highest validation BLEU score)
'''
model.load_state_dict(torch.load('ckpt_seq2seq/seq2seq_100.pt')['state_dict'])
model = model.to(device)

In [ ]:
'''
kickstart full training
'''

best_valid_loss = float('inf')

for epoch in range(2*N_EPOCHS, 4*N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iter, optimizer, criterion, CLIP)

    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    # Create checkpoint at end of each epoch
    state_dict_model = model.state_dict() 
    state = {
        'epoch': epoch,
        'state_dict': state_dict_model,
        'optimizer': optimizer.state_dict()
        }

    if not os.path.isdir("ckpt_seq2seq"):
        os.mkdir("ckpt_seq2seq")
    torch.save(state, "ckpt_seq2seq/seq2seq_"+str(epoch+1)+".pt")

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')

Epoch: 101 | Time: 0m 22s
	Train Loss: 0.244 | Train PPL:   1.277
Epoch: 102 | Time: 0m 22s
	Train Loss: 0.258 | Train PPL:   1.294
Epoch: 103 | Time: 0m 22s
	Train Loss: 0.250 | Train PPL:   1.285
Epoch: 104 | Time: 0m 22s
	Train Loss: 0.249 | Train PPL:   1.283
Epoch: 105 | Time: 0m 22s
	Train Loss: 0.257 | Train PPL:   1.292
Epoch: 106 | Time: 0m 22s
	Train Loss: 0.245 | Train PPL:   1.278
Epoch: 107 | Time: 0m 22s
	Train Loss: 0.253 | Train PPL:   1.288
Epoch: 108 | Time: 0m 22s
	Train Loss: 0.245 | Train PPL:   1.277
Epoch: 109 | Time: 0m 22s
	Train Loss: 0.239 | Train PPL:   1.270
Epoch: 110 | Time: 0m 22s
	Train Loss: 0.234 | Train PPL:   1.263
Epoch: 111 | Time: 0m 22s
	Train Loss: 0.230 | Train PPL:   1.259
Epoch: 112 | Time: 0m 22s
	Train Loss: 0.247 | Train PPL:   1.281
Epoch: 113 | Time: 0m 22s
	Train Loss: 0.269 | Train PPL:   1.309
Epoch: 114 | Time: 0m 22s
	Train Loss: 0.248 | Train PPL:   1.281
Epoch: 115 | Time: 0m 22s
	Train Loss: 0.242 | Train PPL:   1.274
Epoch: 116

# Predictions

In [ ]:
'''
load fields saved during preprocessing
'''
with open("TRG.Field","rb") as f:
     TRG_saved = pickle.load(f)

with open("SRC.Field","rb") as f:
     SRC_saved = pickle.load(f)

'''
hyperparameters (ensure the following hyperparameters match with those used during training of the best model)
'''
INPUT_DIM = len(SRC_saved.vocab)
OUTPUT_DIM = len(TRG_saved.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
N_LAYERS = 1

'''
instantiate the model
'''
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, DEC_HID_DIM, N_LAYERS, DEC_DROPOUT)
model_best = Seq2Seq(enc, dec,device).to(device)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [ ]:
'''
load the checkpoint corresponding to the best epoch (usually epoch with highest validation BLEU score)
'''
model_best.load_state_dict(torch.load('ckpt_seq2seq/seq2seq_200.pt')['state_dict'])
model_best = model_best.to(device)

In [ ]:
test_data = torchtext.legacy.data.TabularDataset(
    path="test_paraphrase.csv", 
    format='csv', skip_header=True, fields=[('src', SRC)]) 

In [ ]:
test_iter = torchtext.legacy.data.Iterator(test_data, batch_size=256, device=device, sort=False, sort_key=None, shuffle=False, sort_within_batch=False)

In [ ]:
'''
generate paraphrases for all the sentences in test data
'''
def generate_paraphrases(model, eval_iter, trg_vocab, attention = True, max_trg_len = 64):
    model.eval()
    all_translation_word_ids = []
    for batch in test_iter:
        src = batch.src
        batch_size = src.shape[1]


        trg_placeholder = torch.Tensor(max_trg_len, batch_size)
        trg_placeholder.fill_(TRG_PAD_IDX)
        trg_placeholder = trg_placeholder.long().to(device)
        if attention == True:
          output,_ = model(src, trg_placeholder, 0) #turn off teacher forcing
        else:
          output = model(src, trg_placeholder, 0) #turn off teacher forcing
        
        output_translate = output[1:]

        # Choose top 1 word from decoder's output, we get the probability and index of the word
        prob, token_id = output_translate.data.topk(1)
        translation_token_id = token_id.squeeze(2).cpu()

        # store gold target sentences to a list 
        all_translation_word_ids.append(translation_token_id)

    all_translation_text = []
    for i in range(len(all_translation_word_ids)):
      cur_translation_batch = all_translation_word_ids[i]
      for j in range(cur_translation_batch.shape[1]):
        trans_convered_strings = convert_itos(trg_vocab, cur_translation_batch[:,j])
        all_translation_text.append(' '.join(trans_convered_strings)) # convert list of words to text
    
    return all_translation_text

In [ ]:
# translate all the sentences in the test set    
test_predictions = generate_paraphrases(model_best, test_iter, TRG_saved, attention = False, max_trg_len = 64)

In [ ]:
test_predictions

['bạn có_thể người_ta đi quyết_tâm nước_mắt , <unk> , theo_dõi cơ_thể bước , ngoài của năm sau khi có_thể gây bệnh <unk> <unk>',
 'điều này cho thấy , bạn tiền về khả_năng <unk> , khả_năng cụ_thể , <unk> của cũng gây ra sự <unk> của doanh_nghiệp thông_qua doanh_nghiệp .',
 'vì_vậy , hãy đảm_bảo chắc_chắn thông_tin vé , mình , hãy , , , bạn bạn bạn bạn có_thể dễ_dàng một <unk> <unk> , <unk> , <unk> , bằng <unk> , bạn mọi nơi bạn có_thể dễ_dàng cần được các loại đồ cổ .',
 'để đạt hiệu_quả , nhiều hoặc cổ của bài nhà_nước đường <unk> , , chủ_nhiệm văn_phòng chính_phủ , đường <unk> , đường <unk> của đường , đường tìm_kiếm nói , những động_viên trong tương_lai … …',
 '<unk> làm các vitamin ở trước máy_tính , trợ_giúp lực_lượng của <unk> , hãy chơi đến <unk> , hãy bán <unk> , <unk> <unk> … của bạn có sự <unk> mà các nước .',
 'đôi_khi đã khiến họ đến hàng ấy , sau khi , , trong game , <unk> lại nỗ_lực để làm các bệnh_viện ở trong vùng miền và cộng_đồng người qua về <unk> : " <unk> nhiều mệt